# Redes neurais artificiais

As redes neurais artificiais são modelos computacionais inspirado no sistema nervoso central dos seres humanos, ou seja, neuronios conectados que emites sinais entre si. As redes neurais são muito utilizadas no aprendizado de máquina e reconhecimento de padrões. Uma rede neural é constituida por inúmeras camadas contendo neurônios interligados que realizando um processamento e propagam o resultado destes processamento para os outros neurônios afim de obter um resultado dado o que foi inserido como entrada para a rede neural. 

## Bibliotecas

In [26]:
import h5py
import torch
import torch.nn as nn
import torchvision
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F
from skorch import NeuralNetBinaryClassifier, NeuralNetClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import GridSearchCV

## Redes MPL

As redes MPL são rede neurais com mais de uma camada totalmente conectadas entre si. Afim de treinar uma rede MPL, será utilizado o conjunto de dados do histórico crediário oferecido aos clientes por uma instituição financeira. A rede neural deve ser capaz de prever se o cliente da instituição financeira irá pagar os não uma dívida dadas as informações do cliente. O problema proposto é um problema de classificação binário, ou o usuário irá pagar ou não a dívida, portanto a rede neural produz como resultado os valores 0 (não irá pagar a dívida) ou 1 (irá pagar a dívida) dadas as 11 característica que recebe como entrada. A arquitetura da rede modelada para este problema possui uma camada linear oculta que recebe como entrada as 11 características do conjunto de dados, produz 64 valores que são passados para a camada linear de saída que produz 1 valor como resultado. A camada oculta utiliza a função de ativação ReLU. Em geral, problemas de classificação binários utilizam a função _sigmoide_ para mapear os resultados em um valor entre 0 e 1. A função _sigmoide_ não está explicitamente definida no passo _forward_ na rede neural, no entanto, a função de perda utilizada _BCEWithLogitsLoss_ aplica a função _sigmoide_ juntamente com a classe _BCELoss_, portanto não é necessário definir a função _sigmoide_ após a computação da camada de saída da rede neural. Além disso, outros parâmetro definidos para o treinamento da rede neural são 200 iterações (parâmetros _max\_epochs_) e taxa de aprendizado (parâmetro _lr_) de 0.01. Em seguida é apresentada a acurácia da rede neural utilizando a função _classification\_report_.

In [4]:
data = pd.read_csv('data/credtrain.txt', sep='\t', header=None)
data_test = pd.read_csv('data/credtest.txt', sep='\t', header=None)

X = np.array(data.iloc[:, :-1]).astype(np.float32)
y = np.array(data.iloc[:, -1]).astype(np.float32)

X_test = np.array(data_test.iloc[:, :-1]).astype(np.float32)
y_test = np.array(data_test.iloc[:, -1])

ss = StandardScaler()
X_norm = ss.fit_transform(X)
X_test_norm = ss.fit_transform(X_test)

In [5]:
class Net(torch.nn.Module):
    def __init__(self, n=64):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(11, n)
        self.fc2 = torch.nn.Linear(n, 1)
        
    def forward(self, x, **kwargs):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = NeuralNetBinaryClassifier(Net, max_epochs=200, lr=0.01, criterion=torch.nn.BCEWithLogitsLoss, verbose=0)
net.fit(X_norm, y)
y_predict = net.predict(X_test_norm)
print(classification_report(y_predict, y_test))

              precision    recall  f1-score   support

           0       0.97      0.85      0.91       349
           1       0.81      0.96      0.88       228

    accuracy                           0.90       577
   macro avg       0.89      0.91      0.90       577
weighted avg       0.91      0.90      0.90       577



Para a otimização dos hiperparâmetros de treinamento da rede neural é utilizada a classe _GridSearch_ da biblioteca _Scikit Learn_, esta classe testa todas as combinações de hiperparâmetros para encontrar os melhores resultados. Foram definidas as taxas de aprendizado 0.1, 0.01 e 0.001, o número de iterações 10, 20, 100, 200, 500, e o número de valor de saída da camada oculta 64, 128 e 256. Em seguida pode ser observado os melhores valores encontrados pelo _GridSearch_.

In [7]:
params = {
    'lr': [0.1, 0.01, 0.001],
    'max_epochs': [10, 20, 100, 200, 500],
    'module__n': [64, 128, 256]
}

grid_search = GridSearchCV(net, params, refit=True, cv=3, scoring='accuracy')
grid_search.fit(X_norm, y)

GridSearchCV(cv=3,
             estimator=<class 'skorch.classifier.NeuralNetBinaryClassifier'>[initialized](
  module_=Net(
    (fc1): Linear(in_features=11, out_features=64, bias=True)
    (fc2): Linear(in_features=64, out_features=1, bias=True)
  ),
),
             param_grid={'lr': [0.1, 0.01, 0.001],
                         'max_epochs': [10, 20, 100, 200, 500],
                         'module__n': [64, 128, 256]},
             scoring='accuracy')

In [16]:
print(f'Parâmetros otimizados: {grid_search.best_params_}')
print(f'Melhor score: {grid_search.best_score_}\n')
y_grid_predict = grid_search.predict(X_test_norm)
print(classification_report(y_grid_predict, y_test))

Parâmetros otimizados: {'lr': 0.01, 'max_epochs': 500, 'module__n': 128}
Melhor score: 0.8866666666666667

              precision    recall  f1-score   support

           0       0.98      0.85      0.91       350
           1       0.81      0.97      0.88       227

    accuracy                           0.90       577
   macro avg       0.89      0.91      0.90       577
weighted avg       0.91      0.90      0.90       577



## Redes convolucionais

Redes neurais convolucionais são um tipo de rede neural bastante utilizado para problemas de análises de imagens digitais. Nesta seção, serão apresentadas duas redes neurais: uma rede neural MPL e uma rede convolucional. Ambas as redes serão treinadas para identificar se uma imagem possui um gato ou não e em seguida será avaliar a acurácia das duas.

### Carga dos dados

In [17]:
def load_dataset():
    train_dataset = h5py.File('data/train_catvnoncat.h5', 'r' )
    train_set_x_orig = np.array(train_dataset['train_set_x'][:]).astype(np.float32) #your train set features
    train_set_y_orig = np.array(train_dataset['train_set_y'][:]) #your train set labels

    test_dataset = h5py.File('data/test_catvnoncat.h5', 'r' )
    test_set_x_orig = np.array(test_dataset['test_set_x'][:]).astype(np.float32) #your test set features
    test_set_y_orig = np.array(test_dataset['test_set_y'][:]) #your test set labels

    classes = np.array(test_dataset['list_classes'][:]) #the list of classes

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

X_train, y_train, X_test, y_test, classes = load_dataset()

### MPL

As imagens do conjunto de dados possuem tamanho 64 $\times$ 64 pixels, sendo cada pixel composto por 3 valores para representar a sua cor: vermelho (R), verde (G) e azul (B). Como redes neurais MPL não suportam dados tridimensionais como entrada deve fazer um preprocessamento nos e linearizar todos os valores em um único vetor, os dados linearizados são representados pelos vetores X_train_flatten e X_test_flatten.

In [19]:
X_train_flatten = X_train.reshape(X_train.shape[0], -1) / 255
X_test_flatten = X_test.reshape(X_test.shape[0], -1) / 255

A arquitetura da rede neural MPL é composta por uma camada oculta que recebe como entrada 12288 valores ($ 64 \times 64 \times 3$), ou seja, o tamanho da imagem linearizada, a camada oculta produz 64 valores para a camada de saída que produz 2 valores. A camada oculta da rede utiliza a função de ativação ReLU. Para mapear os valores produzidos pela camada de saída entre 0 (imagem não possui um gato) e 1 (imagem possui um gato) deve-se aplicar a função _softmax_ para fazer o mapeamento. A função _softmax_ não é definida explicitamente na arquitetura da rede neural após a camada de saída no passo _forwards_, no entanto a função de perda definida pela classe _CrossEntropyLoss_ aplica a função _LogSoftmax_ juntamente com a função _LogSoftmax_. Após o treinamento da rede MPL é utilizada a função _classification\_report_ para verificar a acurácia da rede.

In [20]:
class NetSoftmax(torch.nn.Module):
    def __init__(self):
        super(NetSoftmax, self).__init__()
        self.fc1 = torch.nn.Linear(64 * 64 * 3, 64)
        self.fc2 = torch.nn.Linear(64, 2)
        
    def forward(self, x, **kwargs):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
net_softmax = NeuralNetClassifier(NetSoftmax, max_epochs=250, lr=0.01, criterion=torch.nn.CrossEntropyLoss, verbose=0)
net_softmax.fit(X_train_flatten, y_train)

y_predict = net_softmax.predict(X_test_flatten)

print(classification_report(y_predict, y_test))

              precision    recall  f1-score   support

           0       0.65      0.52      0.58        21
           1       0.70      0.79      0.74        29

    accuracy                           0.68        50
   macro avg       0.67      0.66      0.66        50
weighted avg       0.68      0.68      0.67        50



### Rede convolucional

Para carregar os dados do dataset na rede neural convolucional é necessário definir um dataset torch para refazer o preprocessamento dos dados. O dataset recebe como entrada o vetor de características X e os _targets_ y. Quando um valor é buscado no dataset torch a imagem que é representada por um array numpy é convertida para o formato de imagem da biblioteca _PIL_, esta imagem _PIL_ é então rotacionada, transformada em um tensor torch e os valores são normalizados.

In [22]:
import torchvision.transforms as T

class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.transform = T.Compose([
            T.RandomHorizontalFlip(p=0.5),
            T.ToTensor(),
            T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        X = Image.fromarray(np.uint8(X)).convert('RGB')
        X = self.transform(X)
        return X, y

dataset = CustomDataset(X_train, y_train)
dataset_test = CustomDataset(X_test, y_test)

A arquitetura da rede convolucional consiste de uma camada convolucional, uma camada de _max pooling_ para fazer o _down-sample_ da imagem, uma camada linear oculta que recebe a imagem linearizada e produz 64 valores, e uma camada linear de saída que produz 2 valores. No passo _forward_ a camada convolucional utiliza a função de ativação ReLU, então é aplicado o _max pooling_ para fazer o _down-sample_ da imagem, em seguida a imagem é linearizada pela função _view_ para ser utilizada de entrada para a camada linear ocultada, que utiliza a função de ativacão ReLU, e por fim a imagem passa pela camada de saída. A rede convolucional é treinada com 50 iterações, taxa de aprendizado de 0.001, função de perda definida pela classe _CrossEntropyLoss_ e otimizador _Adam_. Em seguite é apresentada a acurácia da rede treinada utilizando a função _classification\_report_.

In [27]:
class SimpleCNN(torch.nn.Module):
    
    #Our batch shape for input x is (3, 32, 32)
    
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        #Input channels = 3, output channels = 18
        self.conv1 = torch.nn.Conv2d(3, 18, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        #4608 input features, 64 output features (see sizing flow below)
        self.fc1 = torch.nn.Linear(18 * 32 * 32, 64)
        
        #64 input features, 10 output features for our 10 defined classes
        self.fc2 = torch.nn.Linear(64, 2)
        
    def forward(self, x):
        
        #Computes the activation of the first convolution
        #Size changes from (3, 32, 32) to (18, 32, 32)
        x = F.relu(self.conv1(x))
        
        #Size changes from (18, 32, 32) to (18, 16, 16)
        x = self.pool(x)
        
        #Reshape data to input to the input layer of the neural net
        #Size changes from (18, 16, 16) to (1, 4608)
        #Recall that the -1 infers this dimension from the other given dimension
        x = x.view(-1, 18 * 32 * 32)
        
        #Computes the activation of the first fully connected layer
        #Size changes from (1, 4608) to (1, 64)
        x = F.relu(self.fc1(x))
         
        #Computes the second fully connected layer (activation applied later)
        #Size changes from (1, 64) to (1, 10)
        x = self.fc2(x)
        return(x)

In [28]:
net_cnn = NeuralNetClassifier(
    SimpleCNN,
    max_epochs=50,
    lr=0.001,
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    train_split=None,
    verbose=0
)

net_cnn.fit(dataset, y_train)
y_predict = net_cnn.predict(dataset_test)
print(classification_report(y_predict, y_test))

              precision    recall  f1-score   support

           0       0.88      0.68      0.77        22
           1       0.79      0.93      0.85        28

    accuracy                           0.82        50
   macro avg       0.84      0.81      0.81        50
weighted avg       0.83      0.82      0.82        50

